In [10]:
from pathlib import Path
import pandas as pd
from bitstruct import unpack_from as upf
import binascii

In [5]:
def PandUPF(Column, Len, OffBy, OffBi):
    """Extracts a single RAW value from a binary pandas data column"""
    if int(Len[1:]) > 63:
        raise ValueError("PandUPF used for variable larger than 63 bits. Returned value is cast to an Int64")
    Extract = Column.apply(lambda x: upf(Len, x, offset=8*OffBy+OffBi)[0]).astype('Int64')
    return Extract

In [6]:
DIR = Path(r"C:\Users\ucasbwh\Desktop\Proc Dev")

HKfiles = sorted(DIR.rglob("*Unproc_HKTM.pickle"))
print(HKfiles)

RTM = pd.read_pickle(HKfiles[0])
Bin = RTM['RAW'].apply(lambda x: bytearray.fromhex(x))
TM = pd.DataFrame()
TM['DT'] = RTM['DT']
TM['Stat_PIU_Pw'] = PandUPF(Bin, 'u8', 43, 0)
print(TM.head())


[WindowsPath('C:/Users/ucasbwh/Desktop/Proc Dev/191106_212023_Unproc_HKTM.pickle')]
                       DT  Stat_PIU_Pw
0 2019-11-06 21:20:23.587            0
1 2019-11-06 21:20:24.585            0
2 2019-11-06 21:20:25.475            0
3 2019-11-06 21:20:26.587            0
4 2019-11-06 21:20:27.596            0


In [8]:
CamResSeries = Bin.apply(lambda x: x[44:64])
print(CamResSeries)

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
1737    [0, 50, 95, 74, 9, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...
1738    [0, 50, 95, 74, 9, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...
1739    [0, 50, 95, 75, 192, 0, 0, 2, 0, 0, 0, 0, 0, 0...
1740    [0, 50, 95, 75, 192, 0, 0, 2, 0, 0, 0, 0, 0, 0...
1741    [0, 50, 95, 75, 192, 0, 0, 2, 0, 0, 0, 0, 0, 0...
Name: RAW, Length: 1742, dtype: object


In [23]:
CamRes = pd.DataFrame({ 'Bin': CamResSeries,
                        'String': CamResSeries.apply(lambda x: str(binascii.hexlify(x))) })
print(CamRes.head())

Bin  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                        String  
0  b'0000000000000000000000000000000000000000'  
1  b'0000000000000000000000000000000000000000'  
2  b'0000000000000000000000000000000000000000'  
3  b'0000000000000000000000000000000000000000'  
4  b'0000000000000000000000000000000000000000'  


In [20]:
test = CamRes.Bin[0]
end = binascii.hexlify(test)
print(type(end))

<class 'bytes'>


In [35]:
#CamRes['Change'] = CamRes['String'].diff()
#dataframe['changed'] = dataframe['ColumnB'] == dataframe['ColumnB'].shift(1).fillna(dataframe['ColumnB'])
CamRes['BinChange'] = CamRes['Bin'] != CamRes['Bin'].shift(1)
CamRes['CamChange'] = TM['Stat_PIU_Pw'].diff() != 0
print(CamRes) 

Bin  \
0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
...                                                 ...   
1737  [0, 50, 95, 74, 9, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...   
1738  [0, 50, 95, 74, 9, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...   
1739  [0, 50, 95, 75, 192, 0, 0, 2, 0, 0, 0, 0, 0, 0...   
1740  [0, 50, 95, 75, 192, 0, 0, 2, 0, 0, 0, 0, 0, 0...   
1741  [0, 50, 95, 75, 192, 0, 0, 2, 0, 0, 0, 0, 0, 0...   

                                           String  Change  BinChange  \
0     b'0000000000000000000000000000000000000000'    True       True   
1     b'0000000000000000000000000000000000000000'   False      False   
2     b'0000000000000000000000000000000000000000'   False      False   
3     b'0000000000000000000000000000000000000000'   Fal